# Token classification

creating lables

In [ ]:
import os
import sys

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)
from read_data import tags_list  # noqa: E402
from training import get_model, get_dataset, compute_metrics  # noqa: E402

loading test dataset

In [ ]:
file_path = "data/merged.csv"
model_id = "HooshvareLab/bert-base-parsbert-uncased"
tokenizer, model = get_model(model_id)
tokenized_data = get_dataset(file_path, tokenizer)

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
{i: label for i, label in enumerate(tags_list)}

## Train

In [ ]:
import os

os.environ["MLFLOW_EXPERIMENT_NAME"] = "slot-filling-model"
os.environ["MLFLOW_EXPERIMENT_NAME"] = "slot-filling-model"
# os.environ["MLFLOW_FLATTEN_PARAMS"] = "1"
os.environ["MLFLOW_TRACKING_URI"] = "http://127.0.0.1:5000"
# os.environ["HF_MLFLOW_LOG_ARTIFACTS"]="1"

In [ ]:
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback

training_args = TrainingArguments(
    output_dir="ner_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=20,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=1,
    save_total_limit=1,
    load_best_model_at_end=True,
    report_to="mlflow",
    run_name=f"{model_id}",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],
)
trainer.train()

## Inference

Great, now that you've finetuned a model, you can use it for inference!

Grab some text you'd like to run inference on:

In [ ]:
text = """یک دکتر تو جلال آل احمد مشهد می خوام که کار زیبایی بینی انجام بده و خوش اخلاق باشه
یک متخصص زنان و زایمان می خوام که زود نوبت بده و بشه آنلاین ازش نوبت گرفت
سرما خوردم یک پزشک مرد تو شیراز معرفی کن که آنلاین ویزیت کنه
یک روانشناس با سابقه معرفی کن که خوش اخلاق باشه و آنلاین ویزیت کنه تو اصفهان
دلم درد می کنه یک متخصص داخلی مرد که بیمه پاسارگاد قبول کنه نزدیک خیابان شهید رجایی مشهد پیدا کن
یک دندان پزشک برای ترمیم ساده با امتیاز بالا و قیمت مناسب در مشهد
دکتر زنان در محدوده احمد آباد با نظرات رضایت بالا و امتیاز بالا پیدا کن
یک مرکز فیزیوتراپی در محدوده رضاشهر پیدا کن که آسانسور داشته باشه
بهترین آزمایشگاه تهران کجاست؟
چشم پزشکی برای عمل جراحی کجا رضایت مردم بیشتر هست؟
برای اسپاسم شدید عضلات گردن پیش چه دکتری برم و کجا خوبه؟
بوتاکس کدوم دکتر خوبه در"""

The simplest way to try out your finetuned model for inference is to use it in a [pipeline()](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.pipeline). Instantiate a `pipeline` for NER with your model, and pass your text to it:

In [ ]:
from transformers import pipeline

classifier = pipeline("ner", model=model, tokenizer=tokenizer)
for t in text.split("\n"):
    print(t)
    display(classifier(t))